In [9]:
import pandas as pd
import random
from faker import Faker

# Initialisation
fake = Faker()
sexes = ['Homme', 'Femme']
maladie_symptomes = {
    'Paludisme': ['Fièvre', 'Frissons', 'Fatigue', 'Nausée'],
    'Grippe': ['Fièvre', 'Toux', 'Fatigue', 'Maux de tête'],
    'COVID-19': ['Fièvre', 'Toux', 'Fatigue', 'Douleurs musculaires'],
    'Tuberculose': ['Toux', 'Fatigue', 'Fièvre', 'Perte de poids'],
    'Diabète': ['Fatigue', 'Soif excessive', 'Mictions fréquentes'],
    'Hypertension': ['Maux de tête', 'Fatigue', 'Vertiges'],
    'Hépatite': ['Nausée', 'Fatigue', 'Douleurs abdominales', 'Jaunisse']
}

# Générateur de données
def generate_realistic_patient_data(n=1000):
    data = []
    for _ in range(n):
        age = random.randint(1, 100)
        sexe = random.choice(sexes)
        maladie = random.choice(list(maladie_symptomes.keys()))
        symptome_pool = maladie_symptomes[maladie]
        symptomes_patient = random.sample(symptome_pool, k=min(len(symptome_pool), random.randint(2, 4)))
        fumeur = random.choice(['Oui', 'Non'])
        tension = round(random.uniform(9.0, 16.0), 1)
        temperature = round(random.uniform(36.0, 40.5), 1)
        data.append({
            'Age': age,
            'Sexe': sexe,
            'Symptomes': ", ".join(symptomes_patient),
            'Fumeur': fumeur,
            'Tension (cmHg)': tension,
            'Température (°C)': temperature,
            'Maladie': maladie
        })
    return pd.DataFrame(data)

# Génération et sauvegarde
df = generate_realistic_patient_data(1000)
df.to_csv("donnees_patients_realistes.csv", index=False)
print("✅ Fichier CSV généré : donnees_patients_realistes.csv")


✅ Fichier CSV généré : donnees_patients_realistes.csv


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Chargement du fichier CSV
df = pd.read_csv("donnees_patients_realistes.csv")

# Transformation des symptômes en liste
df['Symptomes'] = df['Symptomes'].apply(lambda x: x.split(', '))

# Colonnes numériques
numeriques = ['Age', 'Tension (cmHg)', 'Température (°C)']
X_num = df[numeriques]
scaler = StandardScaler()
X_num_scaled = pd.DataFrame(scaler.fit_transform(X_num), columns=numeriques)

# Colonnes catégorielles (Sexe, Fumeur)
X_cat = df[['Sexe', 'Fumeur']]
ohe = OneHotEncoder(sparse_output=False)
X_cat_encoded = pd.DataFrame(ohe.fit_transform(X_cat), columns=ohe.get_feature_names_out(X_cat.columns))

# Symptômes (encodage multilabel)
mlb = MultiLabelBinarizer()
X_symptoms = pd.DataFrame(mlb.fit_transform(df['Symptomes']), columns=mlb.classes_)

# Construction de X final
X = pd.concat([X_num_scaled, X_cat_encoded, X_symptoms], axis=1)
y = df['Maladie']

# Séparation entraînement/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modèle Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prédictions
y_pred = model.predict(X_test)

# Rapport de performance
print("🔍 Rapport de classification :")
print(classification_report(y_test, y_pred))

import joblib

# Sauvegarde du modèle et des encodeurs
joblib.dump(model, "modele_prediction_maladie.pkl")
joblib.dump(scaler, "scaler.pkl")
joblib.dump(ohe, "onehot_encoder.pkl")
joblib.dump(mlb, "symptom_encoder.pkl")

print("✅ Modèle et encodeurs sauvegardés.")



🔍 Rapport de classification :
              precision    recall  f1-score   support

    COVID-19       0.89      0.94      0.91        34
     Diabète       1.00      1.00      1.00        28
      Grippe       0.86      0.60      0.71        20
Hypertension       0.94      1.00      0.97        33
    Hépatite       0.96      0.96      0.96        26
   Paludisme       0.97      0.94      0.96        36
 Tuberculose       0.85      0.96      0.90        23

    accuracy                           0.93       200
   macro avg       0.92      0.91      0.92       200
weighted avg       0.93      0.93      0.93       200

✅ Modèle et encodeurs sauvegardés.


In [13]:
# import streamlit as st
# import pandas as pd
# import joblib
# import numpy as np



# # Chargement du modèle et des encodeurs
# model = joblib.load("modele_prediction_maladie.pkl")
# scaler = joblib.load("scaler.pkl")
# ohe = joblib.load("onehot_encoder.pkl")
# mlb = joblib.load("symptom_encoder.pkl")

# # Titre
# st.title("🩺 Prédiction de Maladie à partir des Symptômes")

# # Entrée utilisateur
# age = st.slider("Âge", 1, 100, 30)
# sexe = st.selectbox("Sexe", ["Homme", "Femme"])
# fumeur = st.selectbox("Fumeur ?", ["Oui", "Non"])
# tension = st.slider("Tension artérielle (cmHg)", 8.0, 20.0, 12.0)
# temperature = st.slider("Température corporelle (°C)", 35.0, 42.0, 37.0)

# # Liste complète des symptômes possibles (selon entraînement)
# all_symptoms = mlb.classes_
# symptomes = st.multiselect("Symptômes observés", all_symptoms)

# if st.button("Prédire la maladie"):
#     # Préparation des données
#     input_num = scaler.transform([[age, tension, temperature]])
#     input_cat = ohe.transform([[sexe, fumeur]])
#     input_symptoms = mlb.transform([symptomes])
    
#     # Fusion finale
#     input_final = np.concatenate([input_num, input_cat, input_symptoms], axis=1)

#     # Prédiction
#     prediction = model.predict(input_final)[0]
#     st.success(f"✅ Maladie prédite : **{prediction}**")


import subprocess
import time
from IPython.display import IFrame, display



# 1. Lance Streamlit en arrière-plan
cmd = "streamlit run app.py"
process = subprocess.Popen(cmd, shell=True)

# 2. Attendre quelques secondes que Streamlit démarre
time.sleep(5)

# 3. Afficher l'interface Streamlit dans un iframe dans le notebook
display(IFrame(src='http://localhost:8501', width=900, height=700))

# Note : pour arrêter Streamlit, exécute la commande suivante dans une autre cellule :
# process.terminate()





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://192.168.43.171:8502

